# Simple Q & A Bot
## Overview
This is a Simple Q & A Bot that uses Gemini and LangChain to answer questions. The agent is designed to understand user questions and provide accurate answers based on the provided context.

## Motivation
To learn fundamentals of building agents. Creating a Q & A bot is a great way to get started with building agents.
- Demonstrate the use of LangChain and Gemini.
- Show how to build a simple Q & A bot.
- provide a foundation for more complex agents.


## Key Components
1. Language Model: Gemini
2. LangChain: A framework for building agents.
3. Prompt template: for defining the structure of the input and context to the language model.
4. LLMChain: for chaining the language model and prompt template.

## Method Details
1. Setup and Initialization
2. Define the Prompt Template
3. Creating the LLM Chain
4. Implementing Q & A function
5. User Interaction


## Conclusion
This Simple Q & A bot is a great way to get started with building agents. It is a simple and easy to understand example of how to build a Q & A bot using LangChain and Gemini. I can add few more things like:
- Adding memory to maintain context across multiple interactions.
- Adding a knowledge base to provide context to the language model.
- Adding a chat interface to interact with the bot.




##

In [1]:
import os
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
load_dotenv()

True

In [2]:
load_dotenv()
os.environ["GOOGLE_API_KEY"] = os.getenv('GOOGLE_API_KEY')
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash", # Changed model here!
    temperature=0,
    google_api_key=os.getenv('GOOGLE_API_KEY')
)

In [ ]:
template = """"
You are a helpful assistant. Your task is to answer the user's questions to the best of your knowledge.
Question: {question}
Please provide a clear and concise answer:
Answer: """

prompt = PromptTemplate(template=template, input_variables=["question"])